In [1]:
import sys
sys.path.append("/storage/vbutoi/projects/")
sys.path.append("/storage/vbutoi/libraries/")
from ionpy.analysis import ResultsLoader
import pathlib
import seaborn as sns
import pandas as pd
sns.set_context('talk')
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
path_list = [
    # "/storage/vbutoi/scratch/ESE/calibration/09_19_24_ISLES_3D_LTS_BIG_wA",
    # "/storage/vbutoi/scratch/ESE/calibration/09_19_24_ISLES_3D_LTS_BIG_wA_MP",
    "/storage/vbutoi/scratch/ESE/calibration/09_19_24_ISLES_3D_LTS_BIG_noAug",
]

df_list = []
for path in path_list:
    try:
        dfc = rs.load_configs(
            path,
            properties=False,
        )
        df = rs.load_metrics(dfc)
        df_list.append(df)
    except Exception as e:
        print(e)
df = pd.concat(df_list)

In [ ]:
for key in df.keys():
    print(key)

In [4]:
# df.unique_per_col()

In [ ]:
df['path'].unique()

In [6]:
def exp_name(path):
    return str(path).split("/")[-2]
    
df.augment(exp_name)

In [7]:
# Only keep the epochs every 10 to reduce the number of points
df = df[df['epoch'] % 10 == 0]

In [ ]:
# we need to make batch_size and lr cateogorical variables
df['batch_size'] = df['batch_size'].astype('category')
df['lr'] = df['lr'].astype('category')

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='loss',
    hue='exp_name',
    style='seed',
    col='phase',
    kind='line',
    height=12,
    facet_kws={
        "sharey":False,
    }
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.12)
# g.set(ylim=(0.0, 0.25))

In [10]:
def sqrt_loss(loss):
    return loss ** 0.5

df.augment(sqrt_loss)

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='dice_score',
    hue='exp_name',
    style='seed',
    col='phase',
    kind='line',
    height=12,
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# g.set(ylim=(0.0, 0.25))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='sqrt_loss',
    hue='exp_name',
    style='seed',
    col='phase',
    kind='line',
    height=12,
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# g.set(ylim=(0.0, 0.25))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='abs_area_estimation_error',
    hue='exp_name',
    style='seed',
    col='phase',
    kind='line',
    height=12,
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# g.set(ylim=(0.0, 0.25))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='area_estimation_error',
    hue='exp_name',
    style='seed',
    col='phase',
    kind='line',
    height=12,
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# g.set(ylim=(0.0, 0.25))